In [1045]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split,GridSearchCV,learning_curve,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet

In [1084]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [1046]:
df= pd.read_csv('brief_assurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [1047]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [1048]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

selon les descriptions stastique, nous n'avons pas constacte les valeur aberrants ou manquantes. pour max bmi de 53.13 correspondre a un homme de 180cm et 170kg. reste une valeur non aberrants. même conculusion pour nombre d'enfants et age .
donc notre dataframe est prêt à emplois

In [1049]:
df["children"] = df["children"].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   object 
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 73.3+ KB


In [1050]:
X=df.drop('charges',axis=1)
y=df['charges']

In [1051]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)

In [1052]:
quantitative_col = list(X.select_dtypes(include=[float,int]).columns)
categoriel_col = list(X.select_dtypes(include=[object]).columns)

In [1053]:
quantitative_pipeline = make_pipeline(StandardScaler(),PolynomialFeatures())

In [1054]:
categoriel_col

['sex', 'children', 'smoker', 'region']

In [1055]:
preprocessing=ColumnTransformer([("one_hot",OneHotEncoder(),categoriel_col),("scaling",quantitative_pipeline,quantitative_col)])

In [1056]:
preprocessing

ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                 ['sex', 'children', 'smoker', 'region']),
                                ('scaling',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler()),
                                                 ('polynomialfeatures',
                                                  PolynomialFeatures())]),
                                 ['age', 'bmi'])])

In [1057]:
my_pipe_lr = make_pipeline(preprocessing, LinearRegression())

In [1058]:
my_pipe_lasso = make_pipeline(preprocessing, Lasso())

In [1059]:
my_pipe_lasso.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                    ['sex', 'children', 'smoker', 'region']),
                                   ('scaling',
                                    Pipeline(steps=[('standardscaler',
                                                     StandardScaler()),
                                                    ('polynomialfeatures',
                                                     PolynomialFeatures())]),
                                    ['age', 'bmi'])])),
  ('lasso', Lasso())],
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('one_hot', OneHotEncoder(),
                                  ['sex', 'children', 'smoker', 'region']),
                                 ('scaling',
                                  Pipeline(steps=[('standardscaler',
                                                   StandardScaler()),
                        

In [1060]:
my_pipe_lr = make_pipeline(preprocessing, LinearRegression())

In [1061]:
my_pipe_lasso = make_pipeline(preprocessing, Lasso(alpha=4.9))

In [1062]:
my_pipe_ridge= make_pipeline(preprocessing, Ridge(alpha=0.1))

In [1063]:
my_pipe_elasticnet= make_pipeline(preprocessing, ElasticNet(alpha= 0.2, l1_ratio= 0.9))

In [1064]:
my_pipe_lr.fit(X_train, y_train)
my_pipe_lr.score(X_test, y_test)

0.7247420117831209

In [1065]:
my_pipe_lasso.fit(X_train, y_train)
my_pipe_lasso.score(X_test, y_test)

0.7247268059474055

In [1066]:
my_pipe_ridge.fit(X_train, y_train)
my_pipe_ridge.score(X_test, y_test)

0.7248158057991851

In [1067]:
my_pipe_elasticnet.fit(X_train, y_train)
my_pipe_elasticnet.score(X_train, y_train)

0.7602700621970799

In [1068]:
y_pred = my_pipe_ridge.predict(X)
residual =(y_pred-y).abs().sort_values(ascending=False,)
residual

1300    31329.268331
577     25774.785699
543     25196.326167
219     24664.841763
819     24318.594863
            ...     
1305       32.029369
67         22.069095
1310       13.333597
646        10.351579
60          4.058485
Name: charges, Length: 1338, dtype: float64

In [1069]:
to_drop=residual[residual>20000].index

In [1070]:
df[df.index.isin(to_drop)].describe()

,age,bmi,charges
count,13.000000,13.000000,13.000000
mean,36.615385,30.859615,41716.642111
std,12.114306,8.028344,16879.235333
min,20.000000,18.715000,21595.382290
25%,28.000000,23.210000,27375.904780
50%,33.000000,30.360000,35160.134570
75%,45.000000,35.530000,58571.074480
max,55.000000,47.410000,63770.428010


In [1071]:
to_drop

Int64Index([1300, 577, 543, 219, 819, 242, 1027, 1230, 140, 516, 936, 34, 3], dtype='int64')

In [1072]:
df_cleaned=df.drop(to_drop)
df_cleaned

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620
4,32,male,28.88,0,no,northwest,3866.8552
5,31,female,25.74,0,no,southeast,3756.6216
...,...,...,...,...,...,...,...
1333,50,male,30.97,3,no,northwest,10600.5483
1334,18,female,31.92,0,no,northeast,2205.9808
1335,18,female,36.85,0,no,southeast,1629.8335
1336,21,female,25.80,0,no,southwest,2007.9450


In [1073]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   object 
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 73.3+ KB


In [1074]:
X=df.drop('charges',axis=1)
y=df['charges']
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2)
quantitative_col = list(X.select_dtypes(include=[float,int]).columns)
categoriel_col = list(X.select_dtypes(include=[object]).columns)
quantitative_pipeline = make_pipeline(StandardScaler(),PolynomialFeatures())
preprocessing=ColumnTransformer([("one_hot",OneHotEncoder(),categoriel_col),("scaling",quantitative_pipeline,quantitative_col)])


In [1075]:
my_pipe_ridge.fit(X_train, y_train)
my_pipe_ridge.score(X_test, y_test)


0.8059461831201394

In [1076]:
my_pipe_lr.fit(X_train, y_train)
my_pipe_lr.score(X_test, y_test)

0.8049394811316848

meilleur score sans determine parametre alpha 0.765063644178095

In [1077]:
my_pipe_lasso.fit(X_train, y_train)
my_pipe_lasso.score(X_test, y_test)

0.8057183127182372

In [1078]:
my_pipe_elasticnet.fit(X_train, y_train)
my_pipe_elasticnet.score(X_train, y_train)

0.7377083428760434

In [1079]:
import pickle 
pickle_out = open("my_pipe_ridge.pkl", "wb") 
pickle.dump(my_pipe_ridge, pickle_out) 
pickle_out.close()